In [41]:
import pandas as pd
from dotenv import dotenv_values
import numpy as np
import matplotlib.pyplot as plt
import sqlalchemy
import pyodbc


In [42]:
login = dotenv_values('.env')

In [43]:
PASS = login.get('password')
USER = login.get('user')
SERVER = login.get('server')
DATABASE = login.get('dbname')

In [44]:
connection = f"DRIVER={{SQL Server}};SERVER={SERVER};DATABASE={DATABASE};UID={USER};PWD={PASS};"

In [45]:
conn = pyodbc.connect(connection)

In [46]:
query = '''SELECT *
        FROM dbo.LP2_Telco_churn_first_3000'''


In [47]:
df = pd.read_sql_query(query,conn)

df.head()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19484\4188108028.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,conn)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,False,True,False,1,False,None,DSL,False,...,False,False,False,False,Month-to-month,True,Electronic check,29.850000,29.850000,False
1,5575-GNVDE,Male,False,False,False,34,True,False,DSL,True,...,True,False,False,False,One year,False,Mailed check,56.950001,1889.500000,False
2,3668-QPYBK,Male,False,False,False,2,True,False,DSL,True,...,False,False,False,False,Month-to-month,True,Mailed check,53.849998,108.150002,True
3,7795-CFOCW,Male,False,False,False,45,False,None,DSL,True,...,True,True,False,False,One year,False,Bank transfer (automatic),42.299999,1840.750000,False
4,9237-HQITU,Female,False,False,False,2,True,False,Fiber optic,False,...,False,False,False,False,Month-to-month,True,Electronic check,70.699997,151.649994,True
